<a href="https://colab.research.google.com/github/yash-1708/Stock-Analyst-Rating/blob/master/scrapingmarketsmojoBIG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine
import pandas.io.sql as psql
from dateutil import parser
import sqlalchemy
from datetime import datetime

In [4]:
driver = webdriver.Chrome("chromedriver.exe")
broker=[] #List to store name of the broker
company_name=[] #List to store company names 
current_price=[] #List to store rating of the product
predict_date=[]
recomended_buying=[]
target_price=[]
driver.get("https://www.marketsmojo.com/recommendation/")


In [5]:
content = driver.page_source
soup = BeautifulSoup(content,'html.parser')
Alldata=soup.select_one("div.righttexcontent2 div.right-table01 div.scr-result2")
# for row in Alldata.findAll('div',class_='middle-colum02'):
for row in Alldata.findAll('div',attrs={'class':'middle-colum02'}):
    broker.append(row.select("div.colum2-midd01 div.row div.margin-text ul.reco-price li div.text-top0 a.broker-link")[0].get_text())
    company_name.append(row.select("div.colum2-midd02 div.row div.margin-text ul.reco-price li div.text-top0 a.broker-link")[0].get_text().replace('\t','').replace('\n',''))
    current_price.append(row.select("div.colum2-midd03 div.row div.margin-text ul.reco-price li div.text-top0")[0].get_text())
    recomended_buying.append(row.select("div.colum2-midd04 div.row div.margin-text ul.reco-price div.text-top0")[0].get_text())
    target_price.append(row.select("div.colum2-midd05 div.row div.margin-text ul.reco-price li div.text-top0")[0].get_text())
    temp_data=row.select("div.colum2-midd04 div.row div.margin-text ul.reco-price div.colum-text-color-bott")
    temp_data=str(temp_data).split('on ')[1]
    temp_data=temp_data.split('<br/')[0]
    predict_date.append(temp_data)
    #predict_date.append(row.select("div.colum2-midd04 div.row div.margin-text ul.reco-price div.colum-text-color-bott"))
     
for row in Alldata.findAll('div',attrs={'class':'middle-columactive'}):
    broker.append(row.select("div.colum2-midd01 div.row div.margin-text ul.reco-price li div.text-top0 a.broker-link")[0].get_text())
    company_name.append(row.select("div.colum2-midd02 div.row div.margin-text ul.reco-price li div.text-top0 a.broker-link")[0].get_text().replace('\t','').replace('\n',''))
    current_price.append(row.select("div.colum2-midd03 div.row div.margin-text ul.reco-price li div.text-top0")[0].get_text())
    recomended_buying.append(row.select("div.colum2-midd04 div.row div.margin-text ul.reco-price div.text-top0")[0].get_text())
    target_price.append(row.select("div.colum2-midd05 div.row div.margin-text ul.reco-price li div.text-top0")[0].get_text())
    temp_data=row.select("div.colum2-midd04 div.row div.margin-text ul.reco-price div.colum-text-color-bott")
    temp_data=str(temp_data).split('on ')[1]
    temp_data=temp_data.split('<br/')[0]
    predict_date.append(temp_data)

In [6]:
temp_data

'04 Feb 20'

In [7]:
df1 = pd.DataFrame({'broker':broker})  
df2 = pd.DataFrame({'company_name':company_name})
df3 = pd.DataFrame({'current_price':current_price})
df4 = pd.DataFrame({'recomended_buing':recomended_buying})
df5=pd.DataFrame({'target_price':target_price})
df6=pd.DataFrame({'predict_date':predict_date})

In [8]:
rb=df4['recomended_buing'].str.split('Rs.').str[1] 
cp=df3['current_price'].str.split('Rs.').str[1]
tp=df5['target_price'].str.split('Rs.').str[1]
predict_date=df6['predict_date'].str.split('10').str[0]

In [9]:
dfall=pd.DataFrame({'broker':broker, 'company_name':company_name, 'current_price':cp, 'recomended_buying':rb, 'predict_date':predict_date, 'target_price':tp})
dfallclean = dfall[dfall['predict_date']!='']
dfallclean['predict_date'] =  pd.to_datetime(dfallclean['predict_date'], format='%d %b %y')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
dfallclean

,broker,company_name,current_price,recomended_buying,predict_date,target_price
0,Prabhudas Lilladher,Hindalco,150.15,155.75,2020-03-03,220.00
1,Prabhudas Lilladher,Ipca Laboratories,1445.55,1432.89,2020-03-02,1826.00
2,CD Equisearch,Balaji Amines,390.85,422.00,2020-02-28,543.00
3,Geojit Financial Services,Shree Cement,23110.2,23421.21,2020-02-27,20770.00
4,CD Equisearch,MM Forgings,371,400.19,2020-02-26,436.00
...,...,...,...,...,...,...
277,Karvy Stock Broking,Relaxo Footwears,702.45,739.35,2020-02-06,809.00
278,Dolat Capital,Firstsource Solution,40.25,42.81,2020-02-04,62.00
279,Angel Broking,Hawkins Cookers,4988.75,4634.39,2020-02-04,5232.00
280,Geojit Financial Services,Granules India,173.15,146.40,2020-02-04,175.00


In [22]:
database_username = 'smarsproddbuser@mbt-smars-mysql'
database_password = 'modern@1234'
database_ip       = 'mbt-smars-mysql.mysql.database.azure.com'
database_name     = 'mainschema'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))

In [23]:
maxdate = pd.read_sql('SELECT MAX(predict_date) FROM marketsmojorecos', con=database_connection)

In [24]:
maxdate

,MAX(predict_date)
0,2020-03-03


In [14]:
a=maxdate.loc[0,'MAX(predict_date)']

In [15]:
date_object1=maxdate['MAX(predict_date)']
#date_object1.str
mynew=str(a)

In [16]:
date_object1 = datetime.strptime(mynew, '%Y-%m-%d').date()

In [17]:
mask = (dfallclean['predict_date'] > date_object1) 

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  """Entry point for launching an IPython kernel.


In [18]:
dfentry = dfallclean.loc[mask]
dfentry

,broker,company_name,current_price,recomended_buying,predict_date,target_price
0,Prabhudas Lilladher,Hindalco,150.15,155.75,2020-03-03,220.00
1,Prabhudas Lilladher,Ipca Laboratories,1445.55,1432.89,2020-03-02,1826.00
2,CD Equisearch,Balaji Amines,390.85,422.00,2020-02-28,543.00
3,Geojit Financial Services,Shree Cement,23110.2,23421.21,2020-02-27,20770.00
4,CD Equisearch,MM Forgings,371,400.19,2020-02-26,436.00
...,...,...,...,...,...,...
273,Prabhudas Lilladher,Hero MotoCorp,2037.55,2406.37,2020-02-06,2769.00
274,Motilal Oswal,Ajanta Pharma,1446.75,1198.10,2020-02-06,1435.00
275,Prabhudas Lilladher,Max Financial,521.6,484.20,2020-02-06,610.00
276,Motilal Oswal,Bosch,13008.65,13782.55,2020-02-06,15150.00


In [21]:

dfentry.to_sql(con=database_connection, name='marketsmojorecos', if_exists='append')